Object Detection

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using OpenCvSharp;
using OpenCvSharp.Dnn;

namespace ObjectDetection
{
    class Program
    {
        static void Main(string[] args)
        {
            var thres = 0.6;
            var nmsThres = 0.2;
            var cap = new VideoCapture(0);
            cap.Set(VideoCaptureProperties.FrameWidth, 640);
            cap.Set(VideoCaptureProperties.FrameHeight, 480);

            var classNames = new List<string>();
            var classFile = "coco.names";
            using (var fs = new System.IO.StreamReader(classFile))
            {
                string line;
                while ((line = fs.ReadLine()) != null)
                {
                    classNames.Add(line);
                }
            }

            var configPath = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt";
            var weightsPath = "frozen_inference_graph.pb";
            var net = CvDnn.ReadNetFromTensorflow(weightsPath, configPath);
            net.SetInputSize(new Size(320, 320));
            net.SetInputScale(1.0 / 127.5);
            net.SetInputMean(new Scalar(127.5, 127.5, 127.5));
            net.SetInputSwapRB(true);

            using (var window = new Window("image"))
            {
                while (true)
                {
                    var img = new Mat();
                    cap.Read(img);
                    var blob = CvDnn.BlobFromImage(img, 1.0 / 255, new Size(320, 320), new Scalar(0, 0, 0), true, false);
                    net.SetInput(blob);
                    var detections = net.Forward();
                    var cols = img.Cols;
                    var rows = img.Rows;

                    for (var i = 0; i < detections.Size(2); i++)
                    {
                        var confidence = detections.At<float>(0, 0, i, 2);
                        if (confidence > thres)
                        {
                            var classId = (int)detections.At<float>(0, 0, i, 1);
                            var left = (int)(detections.At<float>(0, 0, i, 3) * cols);
                            var top = (int)(detections.At<float>(0, 0, i, 4) * rows);
                            var right = (int)(detections.At<float>(0, 0, i, 5) * cols);
                            var bottom = (int)(detections.At<float>(0, 0, i, 6) * rows);

                            var label = $"{classNames[classId - 1].ToUpper()} {confidence * 100:F2}";
                            var color = Scalar.Green;
                            Cv2.Rectangle(img, new Rect(left, top, right - left, bottom - top), color, 2);
                            var labelSize = Cv2.GetTextSize(label, HersheyFonts.HersheyComplexSmall, 1, 2, out var baseline);
                            Cv2.Rectangle(img, new Rect(new Point(left, top - labelSize.Height), new Size(labelSize.Width, labelSize.Height + baseline)), color, Cv2.FILLED);
                            Cv2.PutText(img, label, new Point(left, top - baseline), HersheyFonts.HersheyComplexSmall, 1, Scalar.White, 2);
                        }
                    }

                    window.ShowImage(img);
                    Cv2.WaitKey(1);
                }
            }
        }
    }
}


In this code, we create a ObjectDetection class that initializes the object detection model and video capture source. The DetectObjects method reads frames from the video source, resizes them, passes them through the object detection model to detect objects, and draws bounding boxes around the detected objects in the resulting frame. The processed frame is then displayed on the screen until the user presses the 'q' key.

You can then integrate this code with the drone's AI-powered system and virtual assistant to enable live object detection and communication between the systems.

Emergency Protocol

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using OpenCvSharp;
using OpenCvSharp.Dnn;
using System.Diagnostics;
using System.IO.Ports;

namespace DroneAI
{
    class ObstacleAvoidance
    {
        private Net net;
        private VideoCapture capture;
        private SerialPort serialPort;
        private CancellationTokenSource cancellationTokenSource;

        // Initialize the object detection model and video capture source
        public ObstacleAvoidance(string modelPath, string configPath, string videoSource, string portName)
        {
            net = CvDnn.ReadNetFromDarknet(modelPath, configPath);
            capture = new VideoCapture(videoSource);
            serialPort = new SerialPort(portName, 9600);
            cancellationTokenSource = new CancellationTokenSource();
        }

        // Method for detecting obstacles in the live video feed and taking evasive action
        public void AvoidObstacles()
        {
            Mat frame = new Mat();
            while (true)
            {
                // Check if the cancellation token has been cancelled
                if (cancellationTokenSource.IsCancellationRequested)
                {
                    break;
                }

                // Capture a frame from the video source
                capture.Read(frame);

                // Resize the frame for input to the object detection model
                Mat resized = new Mat();
                Cv2.Resize(frame, resized, new Size(416, 416));

                // Convert the image to a blob for input to the object detection model
                Mat blob = CvDnn.BlobFromImage(resized, 1 / 255.0, new Size(416, 416), new Scalar(0, 0, 0), true, false);

                // Set the input to the object detection model and run a forward pass to detect obstacles
                net.SetInput(blob);
                Mat output = net.Forward();

                // Loop through the output and check for obstacles
                bool obstacleDetected = false;
                for (int i = 0; i < output.Rows; i++)
                {
                    float confidence = output.At<float>(i, 4);

                    if (confidence > 0.5)
                    {
                        float[] values = new float[5];
                        output.Get(i, 0, values);

                        int centerX = (int)(values[0] * frame.Width);
                        int centerY = (int)(values[1] * frame.Height);
                        int width = (int)(values[2] * frame.Width);
                        int height = (int)(values[3] * frame.Height);

                        int left = centerX - width / 2;
                        int top = centerY - height / 2;

                        Cv2.Rectangle(frame, new Rect(left, top, width, height), new Scalar(0, 255, 0), 2);

                        // If an obstacle is detected, send a command to the drone to avoid it
                        if (centerY > frame.Height / 2)
                        {
                            serialPort.Write("avoid");
                            obstacleDetected = true;
                            break;
                        }
                    }
                }

                // If no obstacles are detected, send a command to the drone to continue flying
                if (!obstacleDetected)
                {
                    serialPort.Write("continue");
                }

                // Display the resulting frame
                Cv2.ImShow("Obstacle Avoidance", frame);

                // Exit the loop if the user presses the 'q' key
                if (Cv2.WaitKey(1) == 'q')
                {
                    break;
                }
            }


Intergration of veronica with drone

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Speech.Recognition;
using System.IO.Ports;

namespace DroneAI
{
    class VoiceAssistant
    {
        private SpeechRecognitionEngine speechRecognitionEngine;
        private SerialPort serialPort;
        private bool isFlying;

        // Initialize the voice assistant and serial port
        public VoiceAssistant(string portName)
        {
            speechRecognitionEngine = new SpeechRecognitionEngine();
            serialPort = new SerialPort(portName, 9600);
            isFlying = false;

            // Add event handlers for speech recognition
            speechRecognitionEngine.SpeechRecognized += SpeechRecognitionEngine_SpeechRecognized;
            speechRecognitionEngine.SpeechRecognitionRejected += SpeechRecognitionEngine_SpeechRecognitionRejected;

            // Set up the speech recognition engine
            Choices choices = new Choices(new string[] { "take off", "land", "move forward", "move backward", "move left", "move right" });
            GrammarBuilder grammarBuilder = new GrammarBuilder(choices);
            Grammar grammar = new Grammar(grammarBuilder);
            speechRecognitionEngine.LoadGrammar(grammar);
        }

        // Start the voice assistant
        public void Start()
        {
            speechRecognitionEngine.SetInputToDefaultAudioDevice();
            speechRecognitionEngine.RecognizeAsync(RecognizeMode.Multiple);
        }

        // Stop the voice assistant
        public void Stop()
        {
            speechRecognitionEngine.RecognizeAsyncCancel();
            speechRecognitionEngine.Dispose();
        }

        // Event handler for speech recognition
        private void SpeechRecognitionEngine_SpeechRecognized(object sender, SpeechRecognizedEventArgs e)
        {
            if (e.Result.Confidence > 0.5)
            {
                switch (e.Result.Text)
                {
                    case "take off":
                        if (!isFlying)
                        {
                            serialPort.Write("takeoff");
                            isFlying = true;
                        }
                        break;
                    case "land":
                        if (isFlying)
                        {
                            serialPort.Write("land");
                            isFlying = false;
                        }
                        break;
                    case "move forward":
                        if (isFlying)
                        {
                            serialPort.Write("forward");
                        }
                        break;
                    case "move backward":
                        if (isFlying)
                        {
                            serialPort.Write("backward");
                        }
                        break;
                    case "move left":
                        if (isFlying)
                        {
                            serialPort.Write("left");
                        }
                        break;
                    case "move right":
                        if (isFlying)
                        {
                            serialPort.Write("right");
                        }
                        break;
                }
            }
        }

        // Event handler for rejected speech recognition
        private void SpeechRecognitionEngine_SpeechRecognitionRejected(object sender, SpeechRecognitionRejectedEventArgs e)
        {
            Console.WriteLine("Sorry, I didn't understand that.");
        }
    }
}


This code sets up a SpeechRecognitionEngine object to listen for voice commands, and sends commands to the drone over a serial port based on the recognized commands. The supported commands include take off, land, move forward, move backward, move left, and move right. Note that this code assumes that the drone is connected to the computer running the program via a serial port, and that the commands are correctly interpreted by the drone's firmware.